**Prod Version of DevEnv**

**This notebook is run using nbconvert after devenv service starts**

In [1]:
!pip install twitter
!pip install feedparser
!pip install schedule
!pip install kafka-python
!pip install redis
!pip install uuid

In [2]:
from time import sleep
from json import dumps, loads, load
from kafka import KafkaProducer
from datetime import datetime, timedelta
from feedparser import parse
from urllib.request import urlopen
from twitter import Twitter
import schedule
from datetime import datetime
import uuid
from pytz import timezone
from bs4 import BeautifulSoup
import urllib.request 
import pandas as pd

In [3]:
tickers = ["MSFT","AAPL"]
feeds = ["http://feeds.reuters.com/reuters/businessNews",
         "https://finance.yahoo.com/news/rssindex/", 
         "https://www.nasdaq.com/feed/rssoutbound",
         "https://news.microsoft.com/feed",
        "https://feeds.a.dj.com/rss/WSJcomUSBusiness.xml",
        "https://feeds.a.dj.com/rss/RSSMarketsMain.xml",
        "http://www.marketwatch.com/rss/topstories",
        "http://www.marketwatch.com/rss/marketpulse",
        "http://www.marketwatch.com/rss/realtimeheadlines",
        "http://feeds.marketwatch.com/marketwatch/bulletins",
        "https://www.cnbc.com/id/10000664/device/rss/rss.html",
        "https://www.cnbc.com/id/10001147/device/rss/rss.html",
        "http://feeds.feedburner.com/TechCrunch/",
        "https://www.ft.com/?format=rss",
        "https://rss.nytimes.com/services/xml/rss/nyt/Business.xml",
        "https://abcnews.go.com/abcnews/moneyheadlines",
        "https://www.cbsnews.com/latest/rss/main",
        "https://markets.businessinsider.com/rss/news",
        "http://newsrss.bbc.co.uk/rss/newsonline_uk_edition/front_page/rss.xml",
        "http://feeds.bbci.co.uk/news/business/rss.xml"]

bootstrapping = True
last_time_step = datetime.now(tz=timezone('US/Pacific'))-timedelta(days=3)
output = {}

producer = KafkaProducer(
    bootstrap_servers=['broker:9092'],
    api_version= (0,11),
    value_serializer=lambda x: dumps(x).encode('utf-8'))

# Authenticate with Twitter & search
# https://pypi.org/project/twitter/
# https://developer.twitter.com/en/apps/17019807
from twitter import *
t = Twitter(auth=OAuth(consumer_key="nDwKHutvaskyGecZkCS1SiYY8", consumer_secret="zlQdJIuFUGGglbMO2kYMk6oNLFztzipxhRANNz5q5gqfn1tGb1", token="73568030-GqfoqkfWRhqhARS9uzz51vg1UGSV7AGRaOxDkAurZ", token_secret="1kiHGfyDfO7gcaJbGFnr3lnLA3vXc8ti9hJhj1lnTTvEm"))

In [4]:
def DownloadStreams():
    # Yahoo Finance Stream for Stock Price
    f_data = {}
    for ticker in tickers:
        try:
            y_finance = load(urlopen("https://query1.finance.yahoo.com/v7/finance/chart/"+ticker+"?range=1h&interval=1h"))
            meta = y_finance['chart']['result'][0]['meta']
            timestamp = meta['regularMarketTime']
            price = meta['regularMarketPrice']
            f_data[ticker] = {"Type": "Finance", "Ticker": ticker, "Timestamp":timestamp, "Price":price}
        except:
            print("Exception in Download Stream for "+ticker)
    
    # News Stream from Reuters for Business
    try:
        if(None == last_time_step):
            last_time_step = datetime.now(tz=timezone('US/Pacific'))-timedelta(days=3)
    except NameError:
        last_time_step = datetime.now(tz=timezone('US/Pacific'))-timedelta(days=3)
        
    feed_data = {}
    for feed in feeds:
        try:
            feed_data[feed] = [{"Type": "News", "Timestamp": datetime.timestamp(pd.to_datetime(l.published)), "Title":l.title, "Summary":BeautifulSoup(l.summary).get_text(strip=True), "Link":l.link} for l in (parse(feed))["entries"] if (pd.to_datetime(l.published)>last_time_step and l.title not in output and "summary" in l and len(l.summary)<500)]
#            feed_data[feed] = [{"Type": "News", "Timestamp": datetime.timestamp(pd.to_datetime(l.published)), "Title":l.title, "Summary":BeautifulSoup(l.summary).get_text(strip=True), "Link":l.link} for l in (parse(feed))["entries"] if (l.title not in output and "summary" in l and len(l.summary)<500)]
        except TypeError:
            print("TypeError")
    
    last_time_step = datetime.now(tz=timezone('US/Pacific'))-timedelta(days=3)
    for feed in feeds:
        for entry in feed_data[feed]:
            output[entry['Title']]=datetime.fromtimestamp(int(entry['Timestamp']),tz=timezone('US/Pacific'))
        
    deletion_keys = [val for val in output if(output[val]<last_time_step)]
    for key in deletion_keys:
        output.pop(key,None)
    
    return f_data, feed_data

def UploadStreamsToKafka(f_data, feed_data, PopularityOnly=False):
    if(not PopularityOnly):
        for key in f_data.keys():
            producer.send('finance_ticker', value=f_data[key])
        for key in feed_data.keys():
            for entry in feed_data[key]:
                #print(entry)
                producer.send('news_feed', value=entry)
    producer.flush()

def HourlyJob():
    # Check, Download & Parse each stream
    f_data, feed_data = DownloadStreams()
    # Upload cleaned events into Kafka Topics
    UploadStreamsToKafka(f_data, feed_data)
    print("Hourly Run Events Complete - Sleeping now")

def MinutelyJob():
    f_data, feed_data = DownloadStreams()
    UploadStreamsToKafka(f_data, feed_data, True)
    print("Minutely Run Events Complete - Sleeping now")



In [5]:
output = {}
HourlyJob()

Processing :http://feeds.reuters.com/reuters/businessNews
Processing :https://finance.yahoo.com/news/rssindex/
Processing :https://www.nasdaq.com/feed/rssoutbound
Processing :https://news.microsoft.com/feed
Processing :https://feeds.a.dj.com/rss/WSJcomUSBusiness.xml
Processing :https://feeds.a.dj.com/rss/RSSMarketsMain.xml
Processing :http://www.marketwatch.com/rss/topstories
Processing :http://www.marketwatch.com/rss/marketpulse
Processing :http://www.marketwatch.com/rss/realtimeheadlines
Processing :http://feeds.marketwatch.com/marketwatch/bulletins
Processing :https://www.cnbc.com/id/10000664/device/rss/rss.html
Processing :https://www.cnbc.com/id/10001147/device/rss/rss.html
Processing :http://feeds.feedburner.com/TechCrunch/
Processing :https://www.ft.com/?format=rss
Processing :https://rss.nytimes.com/services/xml/rss/nyt/Business.xml
Processing :https://abcnews.go.com/abcnews/moneyheadlines
Processing :https://www.cbsnews.com/latest/rss/main
Processing :https://markets.businessi

{'Type': 'News', 'Timestamp': 1582221975.0, 'Title': 'Microsoft announces $1.1B investment plan to drive digital transformation in Mexico, including its first cloud datacenter region', 'Summary': 'The postMicrosoft announces $1.1B investment plan to drive digital transformation in Mexico, including its first cloud datacenter regionappeared first onStories.', 'Link': 'https://news.microsoft.com/es-xl/microsoft-announces-a-1-1-billion-investment-plan-to-drive-digital-transformation-in-country-including-its-first-cloud-datacenter-region/'}
{'Type': 'News', 'Timestamp': 1582208932.0, 'Title': 'How Microsoft 365’s new solution uses machine learning to stop data leaks and insider attacks', 'Summary': 'The postHow Microsoft 365’s new solution uses machine learning to stop data leaks and insider attacksappeared first onStories.', 'Link': 'https://blogs.microsoft.com/ai/insider-risk-management-microsoft-365/?utm_source=stories'}
{'Type': 'News', 'Timestamp': 1582208675.0, 'Title': 'Microsoft at

{'Type': 'News', 'Timestamp': 1582322832.0, 'Title': 'Investors in LatAm get bitten by the hotel investment bug as Ayenda raises $8.7 million', 'Summary': 'Some of Latin America’s leading venture capital investors are now backing hotel chains. In fact, Ayenda, the largest hotel chain in Colombia, has raised $8.7 million in a new round of funding, according to the company. Led by Kaszek Ventures, the round will support the continued expansion of Ayenda’s chain of hotels in Colombia and […]', 'Link': 'http://feedproxy.google.com/~r/Techcrunch/~3/YM43uNA1P38/'}
{'Type': 'News', 'Timestamp': 1582321906.0, 'Title': 'SpaceX said to be seeking around $250 million in funding, boosting valuation to roughly $36 billion', 'Summary': 'SpaceX is looking to raise around $250 million in new funding according to a new report from CNBC’s Michael Sheetz. The additional cash would bring SpaceX’s total valuation to around $36 billion, according to CNBC’s sources — an increase of more than $2.5 billion ver

{'Type': 'News', 'Timestamp': 1582492021.0, 'Title': "Sun's owner reports £68m loss as paper sales fall", 'Summary': 'News Group Newspapers still paying out millions over phone-hacking scandal.', 'Link': 'https://www.bbc.co.uk/news/business-51608099'}
{'Type': 'News', 'Timestamp': 1582502477.0, 'Title': "'Women cocoa farmers paid as little as 23p a day'", 'Summary': 'Campaigning group, Fairtrade, is calling on the government to back a new global equality initiative.', 'Link': 'https://www.bbc.co.uk/news/business-51604345'}
{'Type': 'News', 'Timestamp': 1582502488.0, 'Title': 'Public redundancy bill hits seven-year high', 'Summary': 'Taxpayers are facing a £346m bill after a spate of high-profile business failures on the High Street.', 'Link': 'https://www.bbc.co.uk/news/business-51608107'}
{'Type': 'News', 'Timestamp': 1582422390.0, 'Title': "Brexit: Emmanuel Macron 'not sure' of UK-EU trade deal by end of year", 'Summary': "The French president's doubts come as the UK government is se

In [6]:
MinutelyJob()

Processing :http://feeds.reuters.com/reuters/businessNews
Processing :https://finance.yahoo.com/news/rssindex/
Processing :https://www.nasdaq.com/feed/rssoutbound
Processing :https://news.microsoft.com/feed
Processing :https://feeds.a.dj.com/rss/WSJcomUSBusiness.xml
Processing :https://feeds.a.dj.com/rss/RSSMarketsMain.xml
Processing :http://www.marketwatch.com/rss/topstories
Processing :http://www.marketwatch.com/rss/marketpulse
Processing :http://www.marketwatch.com/rss/realtimeheadlines
Processing :http://feeds.marketwatch.com/marketwatch/bulletins
Processing :https://www.cnbc.com/id/10000664/device/rss/rss.html
Processing :https://www.cnbc.com/id/10001147/device/rss/rss.html
Processing :http://feeds.feedburner.com/TechCrunch/
Processing :https://www.ft.com/?format=rss
Processing :https://rss.nytimes.com/services/xml/rss/nyt/Business.xml
Processing :https://abcnews.go.com/abcnews/moneyheadlines
Processing :https://www.cbsnews.com/latest/rss/main
Processing :https://markets.businessi

In [7]:
schedule.clear()
HourlyJob()

schedule.every(2).minutes.do(HourlyJob)
schedule.every(1).minutes.do(MinutelyJob)
starttime = datetime.now()
endtime = starttime + timedelta(minutes=5)

while True: #datetime.now()<endtime:
    schedule.run_pending()
    sleep(10)
    
schedule.clear()

Processing :http://feeds.reuters.com/reuters/businessNews
Processing :https://finance.yahoo.com/news/rssindex/
Processing :https://www.nasdaq.com/feed/rssoutbound
Processing :https://news.microsoft.com/feed
Processing :https://feeds.a.dj.com/rss/WSJcomUSBusiness.xml
Processing :https://feeds.a.dj.com/rss/RSSMarketsMain.xml
Processing :http://www.marketwatch.com/rss/topstories
Processing :http://www.marketwatch.com/rss/marketpulse
Processing :http://www.marketwatch.com/rss/realtimeheadlines
Processing :http://feeds.marketwatch.com/marketwatch/bulletins
Processing :https://www.cnbc.com/id/10000664/device/rss/rss.html
Processing :https://www.cnbc.com/id/10001147/device/rss/rss.html
Processing :http://feeds.feedburner.com/TechCrunch/
Processing :https://www.ft.com/?format=rss
Processing :https://rss.nytimes.com/services/xml/rss/nyt/Business.xml
Processing :https://abcnews.go.com/abcnews/moneyheadlines
Processing :https://www.cbsnews.com/latest/rss/main
Processing :https://markets.businessi

KeyboardInterrupt: 

In [15]:
output

{'GLOBAL MARKETS-Shares drop, gold surges as coronavirus fears trigger flight for safety': datetime.datetime(2020, 2, 23, 16, 4, 36, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
 'Soft Start Expected For Singapore Stock Market': datetime.datetime(2020, 2, 23, 16, 0, 10, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
 'Oil prices fall 2% on demand concerns as coronavirus spreads outside China': datetime.datetime(2020, 2, 23, 15, 44, 11, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
 'Malaysia Bourse Predicted To Open Under Pressure': datetime.datetime(2020, 2, 23, 15, 30, 10, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
 'More Pain Predicted For South Korea Stock Market': datetime.datetime(2020, 2, 23, 15, 0, 10, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
 'Boston Beer Bursts Seltzer Bubble as Earnings Disappoint': datetime.datetime(2020, 2, 23, 14, 40, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
 'Despite Som

In [16]:
output = {}